In [7]:
#Image Preprocessing

#Imports
import cv2
import numpy as np
import tensorflow as tf
import copy

#Kernels
SharpenKernel = np.array([[0 , -1 , 0] , [-1 , 5 , -1] , [0, -1, 0]])
MorphKernel = cv2.getStructuringElement(cv2.MORPH_RECT , (3,3))

#Images
InputImg = cv2.imread("TestQuestion.png")
InputImg = cv2.resize(InputImg , (450,450))
InputImgGS = cv2.cvtColor(InputImg , cv2.COLOR_BGR2GRAY)
InputImgBlur = cv2.medianBlur(InputImgGS , 5)
InputImgSharpen = cv2.filter2D(InputImgBlur , -1 , SharpenKernel)
InputImgThresh = cv2.adaptiveThreshold(InputImgSharpen, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 3)
InputImgMorph = cv2.morphologyEx(InputImgThresh , cv2.MORPH_CLOSE , MorphKernel)
InputImgMorph = cv2.medianBlur(InputImgMorph , 3)

cv2.imshow("Original" , InputImg)
cv2.imshow("Grey" , InputImgGS)
cv2.imshow("Blur" , InputImgBlur)
cv2.imshow("Sharpen" , InputImgSharpen)
cv2.imshow("Thresh" , InputImgThresh)
cv2.imshow("Morph" , InputImgMorph)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
#Grid Detection

Contours,Heirarchy = cv2.findContours(InputImgMorph , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
PuzzleContours = None

if Contours:
	Contours = sorted(Contours , key=cv2.contourArea , reverse=True)
	for Contour in Contours:
		Perimeter = cv2.arcLength(Contour , True)
		Approx = cv2.approxPolyDP(Contour , Perimeter*0.02 , True)
		if (len(Approx) == 4):
			PuzzleContours = Approx
			break

#Puzzle Cropped Perfectly
if (PuzzleContours is None):
	print("The Puzzle is cropped perfectly")
	x,y,w,h = 0,0,InputImg.shape[1],InputImg.shape[0]
	DetectedBoardImage = InputImgMorph
#Contour Found
else:
	print("Cropping puzzle")
	x,y,w,h = cv2.boundingRect(PuzzleContours)
	DetectedBoardImage = InputImgMorph[y:y+h , x:x+w]
	print(f"Bounds: x={x},y={y},w={w},h={h}")

h,w = DetectedBoardImage.shape
Mask = np.zeros((h+2,w+2) , np.uint8)
cv2.floodFill(DetectedBoardImage , Mask , seedPoint=(250,0) , newVal=0)
Noise,_ = cv2.findContours(DetectedBoardImage , cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
for Dot in Noise:
	Area = cv2.contourArea(Dot)
	if Area <= 75:
		cv2.drawContours(DetectedBoardImage , [Dot] , -1,0,-1)

cv2.imshow("Board" , DetectedBoardImage)
cv2.waitKey(0)
cv2.destroyAllWindows()

Cropping puzzle
Bounds: x=0,y=0,w=450,h=450


In [9]:
#Load Model and feed images

print("Loading Model")
Model = tf.keras.models.load_model("MNISTModel.keras")
print("Model Summary: ")
Model.summary()

Loading Model
Model Summary: 


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_12 (Conv2D)              │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_16 (Activation)      │ (None, 26, 26, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_17 (Activation)      │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 3, 3, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_18 (Activation)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_19 (Activation)      │ (None, 10)             │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 279,968 (1.07 MB)

 Trainable params: 93,322 (364.54 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 186,646 (729.09 KB)

In [10]:
#Cell Segmentation

BoardHeight,BoardWidth = DetectedBoardImage.shape[:2]
CellHeight = BoardHeight//9
CellWidth = BoardWidth//9

Cells = []

for j in range(9):
	CellRow = []
	for i in range(9):
		x1 = i*CellWidth
		y1 = j*CellHeight
		x2 = (i+1)*CellWidth
		y2 = (j+1)*CellHeight

		CellImageTest = DetectedBoardImage[y1:y2 , x1:x2]
		if np.count_nonzero(CellImageTest) < 20:
			Num = 0
		else:
			CellImage = cv2.resize(CellImageTest , (28,28) , interpolation=cv2.INTER_AREA)
			CellImage = CellImage.astype("float32")/255.0
			CellImage = CellImage.reshape(1,28,28,1)
			Prediction = Model.predict(CellImage)
			Num = int(np.argmax(Prediction))

		print(f"Num: {Num}")
		CellRow.append(Num)
		cv2.imshow("Cell" , CellImageTest)
		cv2.waitKey(500)
	Cells.append(CellRow)
print("Puzzle Detected: ")
print(Cells)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 313ms/step
Num: 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Num: 3
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Num: 7
Num: 0
Num: 0
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Num: 6
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Num: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Num: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Num: 5
Num: 0
Num: 0
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Num: 9
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Num: 8
Num: 0
Num: 0
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Num: 6
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Num: 8
Num: 0
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Num: 6
Num: 0
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Num: 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Num: 4
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Num: 8
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Num: 3
Num: 0
Num: 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Num: 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


In [11]:
#Solve Puzzle

def FindNextEmpty(Puzzle):
	for row in range(9):
		for col in range(9):
			if Puzzle[row][col] == 0:
				return row,col
	return None,None

def IsValid(Puzzle , guess , row , col):
	#Row
	if guess in Puzzle[row]:
		return False
	
	#Column
	Col = []
	for i in range(9):
		Col.append(Puzzle[i][col])
	if guess in Col:
		return False
	
	#Grid
	RowStart = (row//3)*3
	ColStart = (col//3)*3

	for r in range(RowStart , RowStart+3):
		for c in range(ColStart , ColStart+3):
			if guess == Puzzle[r][c]:
				return False

	return True

def SolvePuzzle(Puzzle):
	row,col = FindNextEmpty(Puzzle)
	
	if row is None:
		return True
	
	for guess in range(1,10):
		if (IsValid(Puzzle,guess,row,col)):
			Puzzle[row][col] = guess

			if SolvePuzzle(Puzzle):
				return True
			
		Puzzle[row][col] = 0
	return False

print("Original Board: ")
for r in range(9):
	for c in range(9):
		print(Cells[r][c] , end='   ')
	print("\n")
print("\n\n\n")
SolvedCells = copy.deepcopy(Cells)
print("Solved Board: ")
SolvePuzzle(SolvedCells)
for r in range(9):
	for c in range(9):
		print(SolvedCells[r][c] , end = '   ')
	print("\n")
print("\n\n\n")

Original Board: 
5   3   0   0   7   0   0   0   0   

6   0   0   1   9   5   0   0   0   

0   9   8   0   0   0   0   6   0   

8   0   0   0   6   0   0   0   3   

4   0   0   8   0   3   0   0   1   

7   0   0   0   2   0   0   0   6   

0   6   0   0   0   0   2   8   0   

0   0   0   4   1   9   0   0   5   

0   0   0   0   8   0   0   7   9   





Solved Board: 
5   3   4   6   7   8   9   1   2   

6   7   2   1   9   5   3   4   8   

1   9   8   3   4   2   5   6   7   

8   5   9   7   6   1   4   2   3   

4   2   6   8   5   3   7   9   1   

7   1   3   9   2   4   8   5   6   

9   6   1   5   3   7   2   8   4   

2   8   7   4   1   9   6   3   5   

3   4   5   2   8   6   1   7   9   







In [12]:
#Display Output on img

OutputImg = InputImg.copy()
Height,Width = OutputImg.shape[:2]
CellHeight = Height//9
CellWidth = Width//9

for row in range(9):
	for col in range(9):
		if Cells[col][row] == 0:
			val = SolvedCells[col][row]
			x = row*CellWidth
			y = col*CellHeight

			Font = cv2.FONT_HERSHEY_SIMPLEX
			FontScale = 1
			Thickness = 2
			Text = str(val)
			TextSize = cv2.getTextSize(Text,Font,FontScale,Thickness)[0]
			TextX = x + (CellWidth - TextSize[0])//2
			TextY = y + (CellHeight + TextSize[1])//2

			cv2.putText(OutputImg,Text,(TextX,TextY),Font,FontScale,(255,0,0),Thickness,cv2.LINE_AA)

cv2.imshow("Result",OutputImg)
cv2.waitKey(0)
cv2.destroyAllWindows()